In [1]:
import tomllib
import torch
from src.utils.utils import dataclass_from_dict
from src.model.hyperparameters import Hyperparameters
from __future__ import annotations
import pathlib
import sys
from src.model.autoencoder import Autoencoder

sys.path.append(pathlib.Path.cwd().parent)

def load_model_hyperparameters() -> Hyperparameters:
    with (pathlib.Path("/home/paolo/git/spotify-playlist-generator/config/model.toml")).open("rb") as f:
        return dataclass_from_dict(Hyperparameters, tomllib.load(f))
cfg = load_model_hyperparameters()


device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "")
model = Autoencoder.load_from_checkpoint("/home/paolo/git/spotify-playlist-generator/logs/mlruns/749985226233759206/0e707c5e421c47098798cc5c91d9532b/artifacts/model/checkpoints/model_checkpoint/model_checkpoint.ckpt", hyperparam=cfg).eval().half().to(device)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
from torchvision.transforms import v2
from src.model.dataset.transforms import MinMaxNorm

from src.db.schemas.song_embedding import SongEmbedding
import torchaudio.transforms as T


transforms: v2.Compose = v2.Compose([
    T.MelSpectrogram(
        sample_rate=cfg.SAMPLE_RATE,
        n_fft=cfg.N_FFT,
        win_length=cfg.WIN_LENGTH,
        hop_length=cfg.HOP_LENGTH,
        n_mels=cfg.N_MELS,
        normalized=False,
    ),
    MinMaxNorm(),
    v2.ToDtype(torch.float16, scale=False),
])

def preprocess(tensor: torch.Tensor) -> torch.Tensor:
    return transforms(tensor).to(device).unsqueeze(dim=0)

def wrap_prediction_to_song_embedding(tensor: torch.Tensor, song_id: str) -> SongEmbedding:
    print(tensor.shape)
    return SongEmbedding(id=song_id, embedding=tensor.tolist())

def predict_audio(audio: torch.Tensor) -> torch.Tensor:
    crop_frames: int = cfg.SAMPLE_RATE * cfg.CROP_SIZE_SECONDS
    num_frames: int = audio.shape[1]
    last_audio_slice_length: int = num_frames % crop_frames
    print(f"crop_frames: {crop_frames} - num_frames: {num_frames} - last_audio_slice_length: {last_audio_slice_length}")
    
    if last_audio_slice_length == 0:
        print(f"length of last audio is equal to the length of a slice")
        pass
    elif last_audio_slice_length > crop_frames / 2:
        frames_to_add: int = crop_frames - (num_frames % crop_frames)
        print(f"frames_to_add: {frames_to_add}")
        audio = torch.cat([audio, torch.zeros((audio.shape[0], frames_to_add))], dim=1)
    else:
        frames_to_remove: int = num_frames % crop_frames
        print(f"frames_to_remove: {frames_to_remove}")
        audio = audio[:, :num_frames - frames_to_remove]
    
    num_slices: int = audio.shape[1] // crop_frames
    print(f"num_frames: {audio.shape[1]} - num_slices: {num_slices}")
    slices: list[torch.Tensor] = torch.chunk(audio, num_slices, dim=1)
    return torch.stack([model.encoder(preprocess(slice)) for slice in slices]).squeeze().mean(dim=0)


def get_song_embedding(track: dict[str, str]) -> SongEmbedding:
    song_id: str = track.get("song_id")
    audio_path: str = track.get("audio_path")

    audio, _ = torchaudio.load(audio_path)
    num_frames: int = audio.shape[1]
    crop_frames: int = cfg.CROP_SIZE_SECONDS * cfg.SAMPLE_RATE
    if num_frames <= crop_frames:
        frames_to_add: int = crop_frames * (num_frames // crop_frames + 1) - num_frames
        audio = torch.cat([audio, torch.zeros((audio.shape[0], frames_to_add))], dim=1)
        # prediction: torch.Tensor = model.encoder(preprocess(audio))
        # print(f"prediction shape: {prediction.shape}")
        # return wrap_prediction_to_song_embedding(prediction, song_id)

    prediction: torch.Tensor = predict_audio(audio)
    return wrap_prediction_to_song_embedding(prediction, song_id)

In [ ]:
import torchaudio
# get_song_embedding({"song_id": "test", "audio_path": "/home/paolo/git/spotify-playlist-generator/data/raw/songs/1960/4Hhv2vrOTy89HFRcjU3QOx.mp3"})

In [ ]:
audio, _ = torchaudio.load("/home/paolo/git/spotify-playlist-generator/data/raw/songs/1960/4Hhv2vrOTy89HFRcjU3QOx.mp3")

crop_frames = cfg.CROP_SIZE_SECONDS * cfg.SAMPLE_RATE

audio_less = audio[:, :2*crop_frames-(crop_frames//2+10)]
audio_equal = audio[:, :2*crop_frames]

predict_audio(audio_less)
print(f"\n")
predict_audio(audio_equal)
print(f"\n")
predict_audio(audio)

In [ ]:
2159990 - 719990